In [3]:
import torch
import open_clip
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import numpy as np
#!pip install seaborn


In [4]:
import torch
import intel_extension_for_pytorch as ipex

print("XPU available :", torch.xpu.is_available())
if torch.xpu.is_available():
    print("GPU 名稱    :", torch.xpu.get_device_name(0))
    device = torch.device("xpu")
else:
    device = torch.device("cpu")
print("-" * 50)

# 建立測試模型
model = torch.nn.Linear(512, 1000).to(device)
model.eval()

# 產生假資料
data = torch.randn(8, 512).to(device)

# 2025 年最新 IPEX 寫法（只回傳一個值！）
model = ipex.optimize(model, dtype=torch.float32)

# 真正跑一次
with torch.no_grad():
    output = model(data)

print("IPEX + Intel GPU inference 成功")
print("輸出 shape :", output.shape)
print("使用裝置   :", output.device)

XPU available : True
GPU 名稱    : Intel(R) Arc(TM) B580 Graphics
--------------------------------------------------
IPEX + Intel GPU inference 成功
輸出 shape : torch.Size([8, 1000])
使用裝置   : xpu:0


In [5]:
import open_clip
import torch
import intel_extension_for_pytorch as ipex  # Import IPEX for Intel GPU support

# Device detection: Prioritize Intel GPU (XPU) over CPU
if torch.xpu.is_available():
    device = torch.device("xpu")
    print("Using Intel GPU (XPU)")
elif torch.cuda.is_available():
    device = torch.device("cuda")  # Fallback to NVIDIA if available
    print("Using NVIDIA GPU (CUDA)")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Load OpenCLIP model and preprocess (unchanged)
model, _, preprocess = open_clip.create_model_and_transforms(
    model_name="ViT-B-16",
    pretrained="laion400m_e31"
)

# Get tokenizer (unchanged)
tokenizer = open_clip.get_tokenizer("ViT-B-16")

# Move model to device
model.to(device)
model.eval()

# Optional: Optimize with IPEX for better performance on Intel GPU
if device.type == "xpu":
    model = ipex.optimize(model, dtype=torch.bfloat16)  # Use bfloat16 for speed; fallback to float32 if needed
    print("Model optimized with IPEX for Intel GPU")

print("Loaded:", "ViT-B-16 (DataComp-1B)")
print("Device:", next(model.parameters()).device)

Using Intel GPU (XPU)
Model optimized with IPEX for Intel GPU
Loaded: ViT-B-16 (DataComp-1B)
Device: xpu:0


In [6]:
# CELL 3: CIFAR-10 classes + smart prompt ensemble (3 templates = perfect speed/accuracy)
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

test_set = CIFAR10(root='./data', train=False, download=True, transform=preprocess)
test_loader = DataLoader(test_set, batch_size=128)

cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                   'dog', 'frog', 'horse', 'ship', 'truck']

templates = [
    "a photo of a {}.",
    "a blurry photo of a {}.",
    "a photo of a small {}.",
    "a close-up photo of a {}.",
    "a cropped photo of a {}.",
    "a bright photo of a {}.",
    "a low-resolution photo of a {}.",
    "a clean photo of a {}.",
    "a pixelated photo of a {}.",
]

print(f"Using {len(templates)} templates.")



Using 9 templates.


In [7]:
# BUILD TEXT FEATURES (correct tokenizer)
text_features = []

for classname in cifar10_classes:
    texts = [t.format(classname) for t in templates]
    tokens = tokenizer(texts).to(device)
    

    with torch.no_grad():
        emb = model.encode_text(tokens)
        emb = emb / emb.norm(dim=-1, keepdim=True)

    text_features.append(emb.mean(0))

text_features = torch.stack(text_features)
text_features = text_features / text_features.norm(dim=-1, keepdim=True)

RuntimeError: expected scalar type BFloat16 but found Float

In [ ]:
# RUN ZERO-SHOT
all_predictions = []
all_scores = []
true_labels = np.array(test_set.targets)

with torch.no_grad():
    for images, _ in tqdm(test_loader):
        images = images.to(device)

        img_feat = model.encode_image(images)
        img_feat = img_feat / img_feat.norm(dim=-1, keepdim=True)

        sim = (100 * img_feat @ text_features.T).softmax(dim=-1)

        all_scores.append(sim.cpu())
        all_predictions.extend(sim.argmax(dim=1).cpu().numpy())

predictions = np.array(all_predictions)
full_scores = torch.cat(all_scores).numpy()


In [1]:
# CELL 6 – ALL METRICS YOU WILL EVER NEED (research-grade)
from sklearn.metrics import (
    classification_report, confusion_matrix, top_k_accuracy_score,
    balanced_accuracy_score, matthews_corrcoef, precision_recall_fscore_support
)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Convert everything to numpy once
y_true = true_labels
y_pred = predictions
y_scores = full_scores  # shape (10000, 10)

# 1. Top-k accuracies
top1 = np.mean(y_pred == y_true)
top3 = top_k_accuracy_score(y_true, y_scores, k=3)
top5 = top_k_accuracy_score(y_true, y_scores, k=5)

# 2. All classification metrics
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

balanced_acc = balanced_accuracy_score(y_true, y_pred)
mcc = matthews_corrcoef(y_true, y_pred)

# 3. Specificity (macro-averaged)
cm = confusion_matrix(y_true, y_pred)
specificity_per_class = []
for i in range(10):
    tn = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])
    fp = cm[:, i].sum() - cm[i, i]
    specificity_per_class.append(tn / (tn + fp) if (tn + fp) > 0 else 0)
specificity_macro = np.mean(specificity_per_class)

# 4. Per-class accuracy
per_class_acc = cm.diagonal() / cm.sum(axis=1)

# PRINT EVERYTHING BEAUTIFULLY
print("ZERO-SHOT CIFAR-10 – COMPLETE RESULTS")
print("="*60)
print(f"Top-1 Accuracy       : {top1:.4f}  →  {top1*100:05.2f}%")
print(f"Top-3 Accuracy       : {top3:.4f}  →  {top3*100:05.2f}%")
print(f"Top-5 Accuracy       : {top5:.4f}  →  {top5*100:05.2f}%")
print(f"Balanced Accuracy    : {balanced_acc:.4f}")
print(f"Matthews Corr Coef   : {mcc:.4f}")
print(f"Macro F1-score       : {f1_macro:.4f}")
print(f"Weighted F1-score    : {f1_weighted:.4f}")
print(f"Macro Specificity    : {specificity_macro:.4f}")

# Per-class table (sorted)
df_per_class = pd.DataFrame({
    'Class': cifar10_classes,
    'Accuracy': per_class_acc,
    'Specificity': specificity_per_class
}).sort_values('Accuracy')

print("\nPer-class Accuracy (worst → best)")
print(df_per_class.round(4).to_string(index=False))

# Confusion matrix
plt.figure(figsize=(11, 9))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=cifar10_classes, yticklabels=cifar10_classes,
            cbar=False, linewidths=0.5, linecolor='gray')
plt.title('Zero-Shot CLIP – CIFAR-10 Confusion Matrix\n(ViT-B-16 + openai)', fontsize=18)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('True', fontsize=14)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

NameError: name 'true_labels' is not defined